In [2]:
%matplotlib inline

from pathlib import Path

import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
import matplotlib.pylab as plt


## 7.1

#### a.

In [26]:
c1 = ['Stat', 1, 'No']
c2 = ['Other', 1.1, 'Yes']
p = ['IT', 1]

df = pd.DataFrame([c1, c2, p],
                  columns=['전문 분야', '경과 일수', '수강 여부'],
                  index=['client1', 'client2', 'potential'])
df

,전문 분야,경과 일수,수강 여부
client1,Stat,1.0,No
client2,Other,1.1,Yes
potential,IT,1.0,None


In [24]:
df1 = pd.get_dummies(df, drop_first=True)
df1

,경과 일수,전문 분야_Other,전문 분야_Stat,수강 여부_Yes
client1,1.0,0,1,0
client2,1.1,1,0,1
potential,1.0,0,0,0


In [27]:
df2 = pd.get_dummies(df)
df2

,경과 일수,전문 분야_IT,전문 분야_Other,전문 분야_Stat,수강 여부_No,수강 여부_Yes
client1,1.0,0,0,1,1,0
client2,1.1,0,1,0,0,1
potential,1.0,1,0,0,0,0


#### b.

** df1

In [84]:
df1_X = df1[['경과 일수', '전문 분야_Other', '전문 분야_Stat']].iloc[[0,1]]
df1_y = df1['수강 여부_Yes'][[0, 1]]

knn = KNeighborsClassifier(n_neighbors=2).fit(df1_X, df1_y)
distances, indices = knn.kneighbors(df1.iloc[[2],[0,1,2]])
print('Distances',distances)

Distances [[1.         1.00498756]]


** df2

In [93]:
df2_X = df2[['경과 일수', '전문 분야_IT', '전문 분야_Other', '전문 분야_Stat']].iloc[[0,1]]
df2_y = df2['수강 여부_Yes'][[0, 1]]

knn = KNeighborsClassifier(n_neighbors=2).fit(df1_X, df1_y)
distances, indices = knn.kneighbors(df2.iloc[[2],[0,1,2]])
print('Distances',distances)

Distances [[0.1        1.41421356]]


#### c.

* 범주형 변수를 2개의 이진변수로 변환한 데이터세트의 경우(df1), 고객 1과의 유클리드 거리가 더 가까우므로 '코스 수강하지 않음'으로 분류된다.  
* 범주형 변수를 3개의 이진변수로 변환한 데이터세트의 경우(df2), 고객 1과의 유클리드 거리가 더 가까우므로 '코스 수강하지 않음'으로 분류된다.  
* 이 예제의 경우 차이가 없음.

## 7.2

In [4]:
bank_df = pd.read_csv("D:/숙탯 2기/dmba-datasets/UniversalBank.csv")
bank_df.head(10)

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1
5,6,37,13,29,92121,4,0.4,2,155,0,0,0,1,0
6,7,53,27,72,91711,2,1.5,2,0,0,0,0,1,0
7,8,50,24,22,93943,1,0.3,3,0,0,0,0,0,1
8,9,35,10,81,90089,3,0.6,2,104,0,0,0,1,0
9,10,34,9,180,93023,1,8.9,3,0,1,0,0,0,0


In [109]:
bank_df.columns

Index(['ID', 'Age', 'Experience', 'Income', 'ZIP Code', 'Family', 'CCAvg',
       'Education', 'Mortgage', 'Personal Loan', 'Securities Account',
       'CD Account', 'Online', 'CreditCard'],
      dtype='object')

In [113]:
bank_df['Family'].value_counts()

1    1472
2    1296
4    1222
3    1010
Name: Family, dtype: int64

In [114]:
bank_df['Education'].value_counts()

1    2096
3    1501
2    1403
Name: Education, dtype: int64

In [117]:
bank_df['CreditCard'].value_counts()

0    3530
1    1470
Name: CreditCard, dtype: int64

In [5]:
X = ['Age', 'Experience', 'Income', 'ZIP Code', 'Family', 'CCAvg',
       'Education', 'Mortgage', 'Securities Account',
       'CD Account', 'Online', 'CreditCard']
y = 'Personal Loan'

bank_X = pd.get_dummies(bank_df[X], columns=['Family', 'Education'])
bank_y = bank_df[y]
train_X, valid_X, train_y, valid_y = train_test_split(bank_X, bank_y, test_size=0.4, random_state=1)

In [6]:
knn = KNeighborsClassifier(n_neighbors=1).fit(train_X, train_y)
distances, indices = knn.kneighbors(valid_X)
knn.predict(valid_X)

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [13]:
knn.predict(valid_X)

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [18]:
knn.predict(valid_X)[100]

0

#### b.

In [135]:
results = []
for k in range(1, 15):
    knn = KNeighborsClassifier(n_neighbors=k).fit(train_X, train_y)
    results.append({
        'k': k,
        'accuracy': accuracy_score(valid_y, knn.predict(valid_X))
    })

# Convert results to a pandas data frame
results = pd.DataFrame(results).sort_values(by='accuracy', ascending=False)
print(results)

     k  accuracy
9   10    0.9010
10  11    0.9010
3    4    0.9005
11  12    0.9005
5    6    0.8995
12  13    0.8995
1    2    0.8985
4    5    0.8985
7    8    0.8985
8    9    0.8985
13  14    0.8985
6    7    0.8960
2    3    0.8920
0    1    0.8720


최적의 k의 값: 10

#### c.

In [140]:
class_names = ['Personal Loan_0', 'Personal Loan_1']
predicted = ['Personal Loan_1' if p > 0.5 else 'Personal Loan_0' for p in bank_df.Probability]
classificationSummary(bank_df.Class, predicted, class_names=class_names)

AttributeError: 'DataFrame' object has no attribute 'Probability'

잘 모르겠음

#### d.

In [142]:
knn = KNeighborsClassifier(n_neighbors=10).fit(train_X, train_y)
distances, indices = knn.kneighbors(valid_X)
print(knn.predict(valid_X))
print('Distances',distances)
print('Indices', indices)

bank_df1 = pd.concat([bank_X, bank_y], axis=1)
print(bank_df1.iloc[indices[0], :])

[0 0 0 ... 0 0 0]
Distances [[ 21.791971    31.4960315   44.27877595 ...  65.22146886  65.84952543
   67.43448673]
 [  9.45304184  48.97958759  82.83869869 ... 102.78307254 102.82684474
  103.97831505]
 [  5.10783712   5.4          5.92958683 ...  10.42305138  11.56243919
   12.25397895]
 ...
 [ 21.56965461  26.67133293  28.60139857 ...  51.21532974  55.37228549
   55.57958258]
 [  9.56504051  13.65462925  14.18308852 ...  25.78759392  26.36967956
   27.4226184 ]
 [  9.60260381  12.61199826  16.16199245 ...  42.03760222  43.67207346
   45.54217825]]
Indices [[2778  145  362 ... 2024  333 1290]
 [1326 1191  794 ... 1818    0 2650]
 [1623 1621 2152 ... 2689  509 1481]
 ...
 [2481 2705 2551 ...   34 1148  386]
 [ 179  194  620 ...  243  745 2397]
 [1524 1810 1509 ... 1102  541 1837]]
      Age  Experience  Income  ZIP Code  CCAvg  Mortgage  Securities Account  \
2778   56          31      61     92646    1.3         0                   0   
145    59          35     124     90007    7.4  

#### e.

In [148]:
train_X, test_X, train_y, test_y = train_test_split(bank_X, bank_y, test_size=0.2, random_state=1)

train_X, valid_X, train_y, valid_y = train_test_split(train_X, train_y, test_size=0.375, random_state=1)
print(train_X.shape, train_y.shape)
print(valid_X.shape, valid_y.shape)
print(test_X.shape, test_y.shape)

(2500, 17) (2500,)
(1500, 17) (1500,)
(1000, 17) (1000,)


In [149]:
knn = KNeighborsClassifier(n_neighbors=10).fit(train_X, train_y)
distances, indices = knn.kneighbors(test_X)
print(knn.predict(test_X))
print('Distances',distances)
print('Indices', indices)

bank_df1 = pd.concat([bank_X, bank_y], axis=1)
print(bank_df1.iloc[indices[0], :])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

## 7.3

* (e)밖에 못 풀었음

#### a.

In [151]:
housing_df = pd.read_csv('D:\숙탯 2기\dmba-datasets\BostonHousing.csv')
housing_df

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,LSTAT,MEDV,CAT. MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,4.98,24.0,0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,9.14,21.6,0
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,4.03,34.7,1
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,2.94,33.4,1
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,5.33,36.2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273,21.0,9.67,22.4,0
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273,21.0,9.08,20.6,0
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273,21.0,5.64,23.9,0
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273,21.0,6.48,22.0,0


In [156]:
trainData, validData = train_test_split(housing_df.drop(columns=['CAT. MEDV']), test_size=0.4, random_state=26)
print(trainData.shape, validData.shape)

(303, 13) (203, 13)


In [158]:
trainData.columns

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'LSTAT', 'MEDV'],
      dtype='object')

In [168]:
scaler = preprocessing.StandardScaler()
scaler.fit(trainData.drop(columns=['MEDV']))  # Note the use of an array of column names

# Transform the full dataset
housingNorm = pd.concat([pd.DataFrame(scaler.transform(housing_df.drop(columns=['MEDV', 'CAT. MEDV'])), 
                                    columns=['zCRIM', 'zZN', 'zINDUS', 'zCHAS', 'zNOX', 'zRM', 'zAGE', 'zDIS', 'zRAD', 'zTAX',
                                             'zPTRATIO', 'zLSTAT']),
                       housing_df[['MEDV']]], axis=1)
trainNorm = housingNorm.iloc[trainData.index]
validNorm = housingNorm.iloc[validData.index]
# newHouseholdNorm = pd.DataFrame(scaler.transform(newHousehold), columns=['zIncome', 'zLot_Size'])

In [172]:
housingNorm.iloc[validData.index]

,zCRIM,zZN,zINDUS,zCHAS,zNOX,zRM,zAGE,zDIS,zRAD,zTAX,zPTRATIO,zLSTAT,MEDV
130,-0.404598,-0.428676,1.581931,-0.251312,0.635899,0.219914,1.058559,-0.775372,-0.631596,0.184757,1.295994,-0.018637,19.2
187,-0.439804,1.667432,-1.156815,-0.251312,-1.088865,0.672930,-1.078030,0.052374,-0.515542,-0.050967,-1.581530,-0.830444,32.0
129,-0.331684,-0.428676,1.581931,-0.251312,0.635899,-0.928007,0.903305,-0.844066,-0.631596,0.184757,1.295994,0.768486,14.3
32,-0.263411,-0.428676,-0.459140,-0.251312,-0.157308,-0.490371,0.433847,0.152193,-0.631596,-0.600987,1.200076,2.053390,13.2
36,-0.437286,-0.428676,-0.782742,-0.251312,-0.517018,-0.642774,-0.327636,-0.151180,-0.515542,-0.770225,0.336819,-0.181821,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,-0.416463,-0.428676,-0.095458,-0.251312,-0.609251,-0.723870,0.090071,0.333047,-0.631596,-0.782313,0.049067,0.730090,22.5
341,-0.448661,1.201630,-1.441823,-0.251312,-1.042748,1.314703,-0.774915,1.662813,-0.979759,-0.740004,-1.437654,-0.993628,32.7
11,-0.434587,0.153576,-0.499219,-0.251312,-0.286435,-0.407877,0.467116,1.260761,-0.515542,-0.576811,-1.581530,0.073240,18.9
190,-0.438196,1.667432,-1.156815,-0.251312,-1.088865,0.909226,-1.802547,1.386204,-0.515542,-0.050967,-1.581530,-1.047108,37.0


In [185]:
knn = NearestNeighbors(n_neighbors=1)
knn.fit(trainNorm[['zCRIM', 'zZN', 'zINDUS', 'zCHAS', 'zNOX', 'zRM', 'zAGE', 'zDIS', 'zRAD', 'zTAX',
                                             'zPTRATIO', 'zLSTAT']])
distances, indices = knn.kneighbors(validNorm[['zCRIM', 'zZN', 'zINDUS', 'zCHAS', 'zNOX', 'zRM', 'zAGE', 'zDIS', 'zRAD', 'zTAX',
                                             'zPTRATIO', 'zLSTAT']])

print(knn.predict(validNorm[['zCRIM', 'zZN', 'zINDUS', 'zCHAS', 'zNOX', 'zRM', 'zAGE', 'zDIS', 'zRAD', 'zTAX',
                             'zPTRATIO', 'zLSTAT']]))

AttributeError: 'NearestNeighbors' object has no attribute 'predict'

In [187]:
train_X = trainNorm[['zCRIM', 'zZN', 'zINDUS', 'zCHAS', 'zNOX', 'zRM', 'zAGE', 'zDIS', 'zRAD', 'zTAX',
                     'zPTRATIO', 'zLSTAT']]
train_y = trainNorm[['MEDV']]
valid_X = validNorm[['zCRIM', 'zZN', 'zINDUS', 'zCHAS', 'zNOX', 'zRM', 'zAGE', 'zDIS', 'zRAD', 'zTAX',
                     'zPTRATIO', 'zLSTAT']]
valid_y = validNorm[['MEDV']]

results = []
for k in range(1, 15):
    knn = KNeighborsClassifier(n_neighbors=k).fit(train_X, train_y)
    results.append({
        'k': k,
        'accuracy': accuracy_score(valid_y, knn.predict(valid_X))
    })

# Convert results to a pandas data frame
results = pd.DataFrame(results).sort_values(by='accuracy', ascending=False)
print(results)

<ipython-input-187-0603e9b2cfaa>:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  knn = KNeighborsClassifier(n_neighbors=k).fit(train_X, train_y)


ValueError: Unknown label type: 'continuous'

e.

k-최근접이웃 알고리즘은 많은 수의 레코드를 동시에 실시간으로 예측하는 데는 적합하지 않다.